# FVCOM Grid Node Checker Demo
**Author: Jun Sasaki | Created: 2025-09-14 Updated: 2025-09-20**

**Purpose:** Visualize FVCOM grid nodes using xfvcom's core plotting functionality

This notebook demonstrates how to:
- Load FVCOM grid using FvcomInputLoader
- Use FvcomPlotter with FvcomPlotOptions for visualization
- Display node markers and numbers using make_node_marker_post
- Highlight specific nodes of interest
- Export node coordinate information

## Setup and Imports

In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.io.img_tiles import GoogleTiles

# Import xfvcom modules
from xfvcom import (
    FvcomInputLoader,
    FvcomPlotter,
    FvcomPlotConfig,
    FvcomPlotOptions,
    make_node_marker_post,
)

from IPython.core.magic import register_cell_magic
@register_cell_magic
def skip(line, cell):
    print("This cell is skipped.")

# Create output directory for saved figures
output_dir = Path("PNG")
output_dir.mkdir(exist_ok=True)

print("Setup complete!")

## 1. Define Grid File Path and Load Grid
Load the FVCOM grid file to get mesh structure and node coordinates

In [ ]:
# Define path to your grid file
grid_file = Path("~/Github/TB-FVCOM/goto2023/input/TokyoBay18_grd.dat").expanduser()

# UTM zone for Tokyo Bay (adjust for your region)
utm_zone = 54

# Check if file exists
if grid_file.exists():
    print(f"✓ Grid file found: {grid_file}")
    print(f"  File size: {grid_file.stat().st_size / 1024:.1f} KB")
else:
    print(f"✗ Grid file not found: {grid_file}")
    print("  Please update the path to your FVCOM grid file")

# Load grid using FvcomInputLoader
loader = FvcomInputLoader(
    grid_path=grid_file,
    utm_zone=utm_zone,
    add_dummy_time=False,  # We don't need dummy time for this
    add_dummy_siglay=False  # We don't need dummy sigma layers
)

# Get the dataset and grid object
grid_ds = loader.ds
grid_obj = loader.grid

print("\nGrid loaded successfully")
print(f"Number of nodes: {grid_obj.node}")
print(f"Number of elements: {grid_obj.nele}")
print("Coordinate range:")
print(f"  Longitude: {grid_ds.lon.min().values:.3f} - {grid_ds.lon.max().values:.3f}")
print(f"  Latitude: {grid_ds.lat.min().values:.3f} - {grid_ds.lat.max().values:.3f}")

## 2. Initialize Plotter
Create FvcomPlotter instance for visualization

In [ ]:
# Create plotter for visualization
cfg = FvcomPlotConfig()
plotter = FvcomPlotter(grid_ds, cfg)

print("FvcomPlotter initialized")

## 3. Display All Nodes with Markers
Show markers at all node positions using make_node_marker_post

In [ ]:
# Display all nodes with markers (no numbers for performance)
# Create array of all node indices (1-based for FVCOM convention)
all_nodes = np.arange(1, grid_obj.node + 1)

# Define marker styling for all nodes
marker_kwargs = {
    "marker": "o", 
    "color": "lightgray", 
    "markersize": 0.5,  # Small size for all nodes
    "zorder": 4
}

# Create post-processing function for all nodes
pp_all_nodes = make_node_marker_post(
    all_nodes,
    plotter,
    marker_kwargs=marker_kwargs,
    text_kwargs=None,  # No text labels for performance
    index_base=1,
    respect_bounds=True,  # Only show nodes within view bounds
)

# Plot options
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="All Grid Nodes",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_all_nodes, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "all_nodes_markers.png", dpi=300, bbox_inches='tight')
plt.show()
print("All nodes visualization saved to PNG/all_nodes_markers.png")

## 4. Display Specific Nodes with Numbers
Highlight and label specific nodes of interest

In [ ]:
# Define nodes of interest (one-based node numbers)
nodes_of_interest = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 3000]

# Check if nodes are valid
valid_nodes = [n for n in nodes_of_interest if 1 <= n <= grid_obj.node]
invalid_nodes = [n for n in nodes_of_interest if n < 1 or n > grid_obj.node]

print(f"Valid nodes: {valid_nodes}")
if invalid_nodes:
    print(f"Invalid nodes (out of range): {invalid_nodes}")

# Define marker and text styling for specific nodes
mkw = {"marker": "o", "color": "red", "markersize": 5, "zorder": 5}
tkw = {"fontsize": 10, "color": "yellow", "ha": "left", "va": "top",
       "zorder": 6, "clip_on": True}

# Create post-processing function for specific nodes
pp_specific = make_node_marker_post(
    valid_nodes,
    plotter,
    marker_kwargs=mkw,
    text_kwargs=tkw,
    index_base=1,
    respect_bounds=False,  # Show all specified nodes
)

# Plot options with spcified background
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Specific Nodes Highlighted",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_specific, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "specific_nodes.png", dpi=300, bbox_inches='tight')
plt.show()
print("Specific nodes visualization saved to PNG/specific_nodes.png")

## 5. Plot Nodes on Mesh Map (Zoomed View)
Similar to river input checker Section 5 - show nodes on zoomed map with satellite tiles

In [ ]:
# Set map domain for zoomed view (adjust these to your area of interest)
# Example: Focus on central Tokyo Bay
xlim = (139.72, 140.00)
ylim = (35.60, 35.67)

# Select subset of nodes for zoomed view
# You can define specific nodes or use a range
selected_nodes = [100, 200, 300, 500, 1000, 1500, 2000, 2500, 3000]

# Define marker and text styling
mkw = {"marker": "o", "color": "red", "markersize": 5, "zorder": 4}
tkw = {"fontsize": 10, "color": "yellow", "ha": "left", "va": "top",
       "zorder": 5, "clip_on": True}

# Create post-processing function
pp_zoomed = make_node_marker_post(
    selected_nodes,
    plotter,
    marker_kwargs=mkw,
    text_kwargs=tkw,
    index_base=1,
    respect_bounds=True,  # Only show nodes within xlim/ylim
)

# Plot options following river_input_checker style
opts = FvcomPlotOptions(
    figsize=(10, 12),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Node Locations on FVCOM Mesh (Zoomed View)",
    xlim=xlim,
    ylim=ylim,
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=pp_zoomed, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "nodes_map_zoomed.png", dpi=300, bbox_inches='tight')
plt.show()

# Report which nodes are shown vs hidden
nodes_shown = []
nodes_hidden = []
for node_idx_1based in selected_nodes:
    node_idx_0based = node_idx_1based - 1
    if node_idx_0based < len(grid_ds.lon):
        lon = grid_ds.lon.values[node_idx_0based]
        lat = grid_ds.lat.values[node_idx_0based]
        if xlim[0] <= lon <= xlim[1] and ylim[0] <= lat <= ylim[1]:
            nodes_shown.append(node_idx_1based)
        else:
            nodes_hidden.append(node_idx_1based)

print(f"Nodes shown in this view: {nodes_shown}")
print(f"Nodes outside view bounds: {nodes_hidden}")
print("\nTo see all nodes regardless of bounds, set respect_bounds=False")

## 6. Combined Visualization
Show all nodes with specific nodes highlighted

In [ ]:
# Combine all nodes (small blue markers) with highlighted specific nodes (red with numbers)

# Define marker and text styling
mkw = {"marker": "o", "color": "red", "markersize": 4, "zorder": 5}
tkw = {"fontsize": 10, "color": "yellow", "ha": "left", "va": "top",
       "zorder": 6, "clip_on": True}
# Create a combined post-processing function
def combined_post_process(ax):
    # First, plot all nodes as small markers
    all_nodes_subset = np.arange(1, grid_obj.node + 1)
    pp_all = make_node_marker_post(
        all_nodes_subset,
        plotter,
        marker_kwargs={"marker": ".", "color": "lightgray", "markersize": 1, "zorder": 3},
        text_kwargs=None,
        index_base=1,
        respect_bounds=True,
    )
    pp_all(ax)
    
    # Then, highlight specific nodes with numbers
    highlight_nodes = [23, 29, 36, 753, 859, 919, 801, 862, 1507]
    pp_highlight = make_node_marker_post(
        highlight_nodes,
        plotter,
        marker_kwargs=mkw,
        text_kwargs=tkw,
        index_base=1,
        respect_bounds=True,
    )
    pp_highlight(ax)

# Plot options
opts = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=GoogleTiles(style="satellite"),
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="All Nodes with Highlighted Selection",
)

# Create the plot
ax = plotter.plot_2d(da=None, post_process_func=combined_post_process, opts=opts)

# Save figure
ax.figure.savefig(output_dir / "combined_view.png", dpi=300, bbox_inches='tight')
plt.show()
print("Combined visualization saved to PNG/combined_view.png")

## 7. Map View with All Nodes and Node Numbers

In [ ]:
# Create a full map view showing all nodes (or a subset for performance)
# Use every 50th node to avoid overcrowding

# Nodes to be plotted
#display_nodes = np.arange(1, grid_obj.node + 1)  # all nodes
display_nodes = [1443, 1503, 1504, 1506, 1507, 1572, 1573, 1574, 1575, 1576]  # specify 1-based node number list

# Create post-processing function that shows ALL selected markers
pp_node_num = make_node_marker_post(
    display_nodes,
    plotter,
    marker_kwargs={"marker": "o", "color": "red", "markersize": 4, "zorder": 4},
    text_kwargs={"fontsize": 10, "color": "yellow", "ha": "left", "va": "top", "zorder": 5, "clip_on": True},
    index_base=1,
    respect_bounds=True,  # Show all markers regardless of xlim/ylim
    marker_clip_buffer=-0.001,   # in degrees No buffer - strict clipping at boundaries
    text_clip_buffer=-0.001,  # in degrees No buffer - strict clipping at boundaries
)

# Full map extent
# xlim_node_num = (float(grid_ds.lon.min()), float(grid_ds.lon.max()))
# ylim_node_num = (float(grid_ds.lat.min()), float(grid_ds.lat.max()))
# Zoomed map extent
xlim_node_num = (139.87, 139.93)
ylim_node_num = (35.37, 35.44)

# Use GoogleTiles with moderate zoom for faster rendering
from cartopy.io.img_tiles import GoogleTiles

# Create tile provider with moderate resolution
tile_provider_hires = GoogleTiles(style="satellite")

# Plot options for full view
opts_node_num = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=tile_provider_hires,
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Node Locations on FVCOM Mesh",
    xlim=xlim_node_num,
    ylim=ylim_node_num,
)

# Create the full map plot
ax_node_num = plotter.plot_2d(da=None, post_process_func=pp_node_num, opts=opts_node_num)

# Add tiles with moderate zoom level (15 for balance between quality and speed)
if hasattr(ax_node_num, 'add_image'):
    # Clear existing tiles first
    for artist in ax_node_num.artists[:]:
        artist.remove()
    # Add tiles with zoom level 15 (moderate resolution, faster loading)
    ax_node_num.add_image(tile_provider_hires, 15)

# Save figure
ax_node_num.figure.savefig(output_dir / "nodes_num_map.png", dpi=300, bbox_inches='tight')
plt.show()
print("Map showing nodes and numbers; saved to PNG/nodes_num_map.png")
print(f"Total nodes displayed: {len(display_nodes)} out of {grid_obj.node}")

## 8. Calculate Total Area for Selected Nodes (Two Methods)

There are two methods for calculating node areas:
1. **Triangle-based method** (`calculate_node_area`): Sums areas of all triangular elements containing the specified nodes
2. **Median-dual control volume method** (`calculate_node_area_median_dual`): Uses FVCOM's standard median-dual control volumes

The median-dual method is the physically correct approach for FVCOM computations. With the latest implementation, shoreline nodes are closed using true boundary edges (edges that appear only once in the mesh), so control volumes now follow the same coastline/open-boundary definition that FVCOM uses internally. Use this section to compare the historical triangle sum against the FVCOM-accurate control volumes and to confirm that the control-volume totals are internally consistent.

In [ ]:
# Calculate total area for the displayed nodes using BOTH methods
# The display_nodes list was defined in the previous cell
import numpy as np
print(f"Selected nodes (1-based): {display_nodes}")

# Method 1: Triangle-based calculation (original, incorrect for FVCOM physics)
triangle_area_m2 = loader.calculate_node_area(display_nodes, index_base=1)
triangle_area_km2 = triangle_area_m2 / 1e6
triangle_area_ha = triangle_area_m2 / 1e4

print("\n=== Method 1: Triangle-based (simple geometric) ===")
print("Total area of triangular elements containing nodes:")
print(f"  {triangle_area_m2:,.0f} m²")
print(f"  {triangle_area_km2:.4f} km²")
print(f"  {triangle_area_ha:.2f} ha (hectares)")

# Method 2: Median-dual control volume calculation (FVCOM standard, correct)
median_dual_area_m2 = loader.calculate_node_area_median_dual(display_nodes, index_base=1)
median_dual_area_km2 = median_dual_area_m2 / 1e6
median_dual_area_ha = median_dual_area_m2 / 1e4

print("\n=== Method 2: Median-dual control volume (FVCOM physics) ===")
print("Total area of median-dual control volumes for nodes:")
print(f"  {median_dual_area_m2:,.0f} m²")
print(f"  {median_dual_area_km2:.4f} km²")
print(f"  {median_dual_area_ha:.2f} ha (hectares)")

# Compare the two methods
ratio = median_dual_area_m2 / triangle_area_m2 if triangle_area_m2 else float('nan')
print("\n=== Comparison ===")
print(f"Difference: {abs(triangle_area_m2 - median_dual_area_m2):,.0f} m²")
print(f"Ratio (median-dual / triangle): {ratio:.6f}")

# Sanity check: sum of individual control volumes matches the aggregated value
subset_cv_sum = sum(
    loader.calculate_node_area_median_dual([node], index_base=1)
    for node in display_nodes
)
cv_error = abs(subset_cv_sum - median_dual_area_m2)
print("\n=== Sanity check ===")
print(f"Sum of individual control volumes: {subset_cv_sum:,.0f} m²")
print(f"Consistency error: {cv_error:.6e} m²")
print(f"Subset consistency OK: {np.isclose(subset_cv_sum, median_dual_area_m2, rtol=1e-12, atol=1e-8)}")

# Compare with total mesh area
print("\n=== For reference ===")
# Total mesh area using median-dual (correct)
total_md_area = loader.calculate_node_area_median_dual(None, index_base=1)  # All nodes
selection_fraction = 100 * median_dual_area_m2 / total_md_area if total_md_area else float('nan')
print(f"  Total mesh area (median-dual): {total_md_area/1e6:.2f} km²")
print(f"  Selected area percentage: {selection_fraction:.4f}%")

## 9. Alternative: Using Standalone Function

You can also calculate node areas directly from a grid file without loading the full dataset.

In [ ]:
# Alternative method: Use the standalone function for direct calculation
import numpy as np
from xfvcom import calculate_node_area

# Example with a custom list of nodes
custom_nodes = [1000, 1001, 1002, 1003, 1004]

# Calculate area directly from grid file (triangle-based)
area_direct = calculate_node_area(
    grid_file=grid_file,
    node_indices=custom_nodes,
    utm_zone=utm_zone,
    index_base=1  # Using 1-based indexing (FVCOM convention)
)

print(f"Custom node list: {custom_nodes}")
print(f"Triangle-area total: {area_direct:,.0f} m² ({area_direct/1e6:.4f} km²)")

# Median-dual calculation for the same nodes using the loader (physically correct)
md_area_direct = loader.calculate_node_area_median_dual(custom_nodes, index_base=1)
print(f"Median-dual total: {md_area_direct:,.0f} m² ({md_area_direct/1e6:.4f} km²)")

# Example with zero-based indexing
# If you have node indices from a zero-based system (like Python arrays)
zero_based_nodes = [999, 1000, 1001, 1002, 1003]  # Same nodes as above but 0-based
area_zero_based = calculate_node_area(
    grid_file=grid_file,
    node_indices=zero_based_nodes,
    utm_zone=utm_zone,
    index_base=0  # Using 0-based indexing
)
md_area_zero_based = loader.calculate_node_area_median_dual(zero_based_nodes, index_base=0)

print(f"\nZero-based node list: {zero_based_nodes}")
print(f"Triangle-area total: {area_zero_based:,.0f} m²")
print(f"Median-dual total: {md_area_zero_based:,.0f} m²")
print(f"Triangle areas match: {np.isclose(area_direct, area_zero_based)}")
print(f"Median-dual areas match: {np.isclose(md_area_direct, md_area_zero_based)}")

## 10. Visualize Median-Dual Control Volumes

Display the FVCOM median-dual control volume polygons for selected nodes. These are the actual areas used in FVCOM physics calculations.

In [ ]:
# Get median-dual control volume polygons for visualization
import numpy as np
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

# Get control volume polygons from the loader
control_volumes = loader.get_node_control_volumes(display_nodes, index_base=1)

print(f"Retrieved {len(control_volumes)} control volume polygons")

def plot_with_control_volumes(ax):
    """Display nodes with their median-dual control volume polygons."""
    # Add node markers and numbers
    pp_node_num(ax)
    
    # Create polygon patches for each control volume
    patches = []
    for poly_coords in control_volumes:
        if len(poly_coords) < 3:
            continue
            
        # Check if polygon is within bounds
        if opts_cv.xlim and opts_cv.ylim:
            xs = [p[0] for p in poly_coords]
            ys = [p[1] for p in poly_coords]
            
            # Simple bounds check
            if (max(xs) < opts_cv.xlim[0] - 0.01 or 
                min(xs) > opts_cv.xlim[1] + 0.01 or
                max(ys) < opts_cv.ylim[0] - 0.01 or 
                min(ys) > opts_cv.ylim[1] + 0.01):
                continue
        
        # Transform coordinates if using cartopy
        if hasattr(ax, 'projection'):
            transformed = ax.projection.transform_points(
                ccrs.PlateCarree(),
                np.array([p[0] for p in poly_coords]),
                np.array([p[1] for p in poly_coords])
            )
            poly_coords_transformed = [(x, y) for x, y in transformed[:, :2]]
            patches.append(Polygon(poly_coords_transformed))
        else:
            patches.append(Polygon(poly_coords))
    
    # Create patch collection with semi-transparent blue fill
    # Blue indicates these are control volumes (different from red triangles)
    if patches:
        pc = PatchCollection(patches, 
                           facecolor='blue', 
                           edgecolor='darkblue',
                           alpha=0.3, 
                           linewidth=1.5,
                           zorder=3)
        ax.add_collection(pc)
        print(f"Added {len(patches)} control volume patches to plot")

# Plot options for control volume view
opts_cv = FvcomPlotOptions(
    figsize=(12, 10),
    add_tiles=True,
    tile_provider=tile_provider_hires,
    mesh_color="lightgray",
    mesh_linewidth=0.2,
    title="Median-Dual Control Volumes (FVCOM Physics)",
    xlim=xlim_node_num,
    ylim=ylim_node_num,
)

# Create the plot
ax_cv = plotter.plot_2d(
    da=None, 
    post_process_func=plot_with_control_volumes, 
    opts=opts_cv
)

# Add high-res tiles
if hasattr(ax_cv, 'add_image'):
    for artist in ax_cv.artists[:]:
        artist.remove()
    ax_cv.add_image(tile_provider_hires, 15)

# Save figure
ax_cv.figure.savefig(output_dir / "nodes_with_control_volumes.png", dpi=300, bbox_inches='tight')
plt.show()

subset_cv_sum = sum(
    loader.calculate_node_area_median_dual([node], index_base=1)
    for node in display_nodes
)
print(f"\nNumber of control volume polygons: {len(control_volumes)}")
print(f"Total control volume area: {median_dual_area_m2:,.0f} m²")
print(f"Subset sum check: {subset_cv_sum:,.0f} m² (match: {np.isclose(subset_cv_sum, median_dual_area_m2)})")
print(f"Compare to triangle area: {triangle_area_m2:,.0f} m²")
print(f"Ratio (CV/triangle): {median_dual_area_m2/triangle_area_m2:.6f}")
print("\nNote: Blue polygons show median-dual control volumes (correct for FVCOM).")
print("      Red polygons (previous section) show triangular elements (geometric only).")
print("\nFigure saved to PNG/nodes_with_control_volumes.png")

## 11. Calculate Element Areas for Selected Cells

Use the new element-area helper to inspect specific triangular cells. This mirrors the node-based area checks but works directly with FVCOM's element numbering.

In [ ]:
# Calculate areas for selected elements
import numpy as np
from xfvcom.grid import calculate_element_area

# Pick a handful of element numbers (1-based) near the study region
selected_elements = list(range(1, 6))
print(f"Selected elements (1-based): {selected_elements}")

# Loader method (returns one value per element)
loader_element_areas = loader.calculate_element_area(selected_elements, index_base=1)
print("=== Element areas via loader ===")
for elem, area in zip(selected_elements, loader_element_areas):
    print(f"  Element {elem}: {area:,.2f} m² ({area/1e6:.6f} km²)")

# Convenience helper that loads directly from disk (triangle-based)
areas_direct = calculate_element_area(
    grid_file=grid_file,
    element_indices=selected_elements,
    utm_zone=utm_zone,
    index_base=1,
)
print("Areas match loader: ", np.allclose(loader_element_areas, areas_direct))

# Aggregate comparisons
all_element_areas = loader.calculate_element_area(None, index_base=1)
print(f"Total mesh area from element sum: {all_element_areas.sum()/1e6:.2f} km²")
print(f"Subset share of total: {100 * loader_element_areas.sum() / all_element_areas.sum():.6f}%")